Con este codigo se lee el archivo flights y con la columna scheduled_departure se realiza un calculo agregando un retraso aleatorio entre 0 y 30 minutos para almacenarlo en la columna actual_departure  en las filas vacias en un archivo auxiliar intemedio llamado "archivo_modificado"

In [2]:
import pandas as pd
import random
from datetime import timedelta

# Cargar el archivo CSV
df = pd.read_csv('C:\\Users\\Invite\\Documents\\travel_sqlite\\flights.csv')

# Asegurarse de que 'scheduled_departure' esté en formato datetime
df['scheduled_departure'] = pd.to_datetime(df['scheduled_departure'], format='ISO8601')

# Crear una función que modifique actual_departure dependiendo de scheduled_departure
def update_actual_departure(row):
    # Si 'actual_departure' está vacío o es '\N', asignar un valor aleatorio
    if pd.isna(row['actual_departure']) or row['actual_departure'] == "\\N":
        # Generar un retraso aleatorio entre 0 y 30 minutos
        delay_minutes = random.randint(0, 30)  # Este valor cambia aleatoriamente para cada fila
        actual_departure = row['scheduled_departure'] + timedelta(minutes=delay_minutes)
        return actual_departure
    return row['actual_departure']  # Si ya tiene un valor de actual_departure, no lo cambiamos



# Aplicar la función a cada fila del DataFrame
df['actual_departure'] = df.apply(update_actual_departure, axis=1)

df['actual_departure'] = pd.to_datetime(df['actual_departure'], format='ISO8601')

# Obtener la diferencia en minutos
df['departure_delay_minutes'] = (df['actual_departure'] - df['scheduled_departure']).dt.total_seconds() / 60

# Imprimir el resultado para verificar
print(df[['flight_id', 'scheduled_departure','scheduled_arrival', 'actual_departure','actual_arrival','departure_delay_minutes']])

# Guardar el archivo modificado
df.to_csv('C:\\Users\\Invite\\Documents\\travel_sqlite\\archivo_modificado.csv', index=False)

       flight_id       scheduled_departure       scheduled_arrival  \
0           1185 2017-09-10 09:50:00+03:00  2017-09-10 14:55:00+03   
1           3979 2017-08-25 14:50:00+03:00  2017-08-25 17:35:00+03   
2           4739 2017-09-05 12:30:00+03:00  2017-09-05 14:15:00+03   
3           5502 2017-09-12 09:50:00+03:00  2017-09-12 11:20:00+03   
4           6938 2017-09-04 12:25:00+03:00  2017-09-04 13:20:00+03   
...          ...                       ...                     ...   
33116      33117 2017-08-02 19:25:00+03:00  2017-08-02 20:10:00+03   
33117      33118 2017-07-28 19:25:00+03:00  2017-07-28 20:10:00+03   
33118      33119 2017-09-08 19:25:00+03:00  2017-09-08 20:10:00+03   
33119      33120 2017-08-01 19:25:00+03:00  2017-08-01 20:10:00+03   
33120      33121 2017-08-26 19:25:00+03:00  2017-08-26 20:10:00+03   

               actual_departure          actual_arrival  \
0     2017-09-10 10:12:00+03:00                      \N   
1     2017-08-25 14:53:00+03:00          

Usando como fuente el archivo_modificado resultante del codigo anterior se realiza un procedimiento similar pero en la columna actual_arrival, ademas se agregan nuevas columnas de tiempos de retraso y duracion del vuelo

In [ ]:
df_mod = pd.read_csv('C:\\Users\\Invite\\Documents\\travel_sqlite\\archivo_modificado.csv')

# Asegurarse de que 'scheduled_departure' esté en formato datetime
df_mod['scheduled_departure'] = pd.to_datetime(df_mod['scheduled_departure'], format='ISO8601')
# Asegurarse de que 'actual_departure' esté en formato datetime
df_mod['actual_departure'] = pd.to_datetime(df_mod['actual_departure'], format='ISO8601')
# Asegurarse de que 'scheduled_arrival' esté en formato datetime
df_mod['scheduled_arrival'] = pd.to_datetime(df_mod['scheduled_arrival'], format='ISO8601')


# Crear una función que modifique actual_arrival dependiendo de scheduled_arrival
def update_actual_arrival(row):
    # Si 'actual_departure' está vacío o es '\N', asignar un valor aleatorio
    if pd.isna(row['actual_arrival']) or row['actual_arrival'] == "\\N":
        # Generar un retraso aleatorio entre -10 y 17 minutos
        delay_minutes = random.randint(-10, 17)  # Este valor cambia aleatoriamente para cada fila, tambien se suma el retraso en la salida
        actual_arrival = row['scheduled_arrival'] + timedelta(minutes=delay_minutes)+timedelta(minutes=row['departure_delay_minutes'])
        return actual_arrival
    return row['actual_arrival']  # Si ya tiene un valor de actual_departure, no lo cambiamos

# Aplicar la función a cada fila del DataFrame
df_mod['actual_arrival'] = df_mod.apply(update_actual_arrival, axis=1)
# Asegurarse de que 'actual_departure' esté en formato datetime
df_mod['actual_arrival'] = pd.to_datetime(df_mod['actual_arrival'], format='ISO8601')

# Obtener la diferencia en minutos
df_mod['arrival_delay_minutes'] = (df_mod['actual_arrival'] - df_mod['scheduled_arrival']).dt.total_seconds() / 60

# Obtener la duracion del vuelo en minutos
df_mod['duration_mm']=(df_mod['actual_arrival']-df_mod['actual_departure']).dt.total_seconds()/60


# Calcular la diferencia entre actual_arrival y actual_departure
df_mod['duration'] = df_mod['actual_arrival'] - df_mod['actual_departure']

# Convertir la diferencia a formato HH:MM
df_mod['duration_hh_mm'] = df_mod['duration'].apply(lambda x: f"{x.seconds // 3600:02}:{(x.seconds % 3600) // 60:02}")

#df_mod['flight_time2'] = pd.to_datetime(df_mod['flight_time'],format='%H:%M' ).dt.time

# Imprimir el resultado para verificar
print(df_mod[['flight_id', 'scheduled_departure','scheduled_arrival', 'actual_departure','actual_arrival','departure_delay_minutes','arrival_delay_minutes']])

df_mod.to_csv('C:\\Users\\Invite\\Documents\\travel_sqlite\\flights_modificado.csv', index=False)

       flight_id       scheduled_departure         scheduled_arrival  \
0           1185 2017-09-10 09:50:00+03:00 2017-09-10 14:55:00+03:00   
1           3979 2017-08-25 14:50:00+03:00 2017-08-25 17:35:00+03:00   
2           4739 2017-09-05 12:30:00+03:00 2017-09-05 14:15:00+03:00   
3           5502 2017-09-12 09:50:00+03:00 2017-09-12 11:20:00+03:00   
4           6938 2017-09-04 12:25:00+03:00 2017-09-04 13:20:00+03:00   
...          ...                       ...                       ...   
33116      33117 2017-08-02 19:25:00+03:00 2017-08-02 20:10:00+03:00   
33117      33118 2017-07-28 19:25:00+03:00 2017-07-28 20:10:00+03:00   
33118      33119 2017-09-08 19:25:00+03:00 2017-09-08 20:10:00+03:00   
33119      33120 2017-08-01 19:25:00+03:00 2017-08-01 20:10:00+03:00   
33120      33121 2017-08-26 19:25:00+03:00 2017-08-26 20:10:00+03:00   

               actual_departure            actual_arrival  \
0     2017-09-10 10:12:00+03:00 2017-09-10 15:22:00+03:00   
1     2017-08

Posterior a la aplicacion de este codigo se debe hacer tambien el borrado de los tiempos para los vuelos cancelados ya que estos no tendrian ni salida ni llegada reales. Este paso se realiza usando google sheets.